### introduction
---
    - gradient Descent
    - First_order iterative optimziation algorithm for finding a local minimum of a differnetiale function
---
    - generalization 
    - under-fitting vs over-fitting
    - cross validation
    - Bias-variance tradeoff
    - Bootstrapping
    - Bagging and boosting
---
    Cross-validation
    - k-fold validation
    - 학습데이터를 나누어서 test_date와 분리한다. 
---
    Bias and variance
    - variance: 일관성을 뜻하며 , low 일수록 일관성이 높다는 것을 뜻한다.
    - bias: Cost= bias^2 +variance+noise -> 이런식으로 3가지로 이루어져있기 때문에  cost를 줄이기 위해 고려해야 할 것은 3가지가 있다.
---
    Booststrapping
    - any test or metric that uses random sampling whith replacement
    - 기본적으로 학습데이터가 고정되어있을 떄 subsampling을 통해 여러 개로 분류해서 trainging 한다
---
    Bagging vs boosting
    -Bagging( Bootstrapping aggregating)
        - multiple models are being trained with Booststrapping
        - Base classifiers are fitted on random subset where individual predictions are aggregated
    - boosting
        - 분류하기 어려운 sample에 집중한다.
        - 여러개의 모델을 만들어서 합친다. weak learners 들을 sequence하게 합친다.(하나의 강력한 모델을 만든다, bagging 은 여러개의 모델을 병렬적으로 실행)
---
    gradient descent Methods
---
    Batch-size Matters
    - momentum
    - adam
---
    Regularization
    - Early stopping
        - 빨리멈춰서 error율을 확인한다
    - parameter norm penalty
        -숫자들을 작게하자 
    - data augmentation
        - 더 많은 데이터가 더 좋다. (데이터 증가시키기)
        - mnist와 같이 label이 바뀌는 경우엔 rotate ,flip, crop 등이 label을 바꿀수 있기 떄문에 안되지만 목적에 따라서 사용가능 하다.
    - noise robustness
        - add random noises input or weight ( 일부러 노이즈를 계속 추가하는 방법)
    - label smoothing
        - 학습데이터 몇개를 뽑아서 섞어준다.
    - Dropout
        -randomly set units zero
    - Batch normalization
        - batch norm, layer norm, instance norm, group norm
        - 분류문제에 있어서는 일반적으로 성능이 좋다 ( 확실하진 않다.)    


In [3]:
!pip install matplotlib==3.3.0
!pip install torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
print('done')

In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from IPython import get_ipython

# %%
get_ipython().system('pip install matplotlib==3.1.3')
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
get_ipython().magic('matplotlib inline')
get_ipython().magic("config InlineBackend.figure_format='retina'")
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))
n_data = 10000
x_numpy = -3+6*np.random.rand(n_data,1)
y_numpy = np.exp(-(x_numpy**2))*np.cos(10*x_numpy) + 3e-2*np.random.randn(n_data,1)
plt.figure(figsize=(8,5))
plt.plot(x_numpy,y_numpy,'r.',ms=2)
plt.show()
x_torch = torch.Tensor(x_numpy).to(device)
y_torch = torch.Tensor(y_numpy).to(device)
print ("Done.")

# %% [markdown]
# ### define model

# %%
class Model(nn.Module):
    def __init__ (self,name='mlp',xdim=1,hdims=[16,16],ydim=1):
        super(Model,self).__init__()
        self.name=name
        self.xdim=xdim
        self.hdims=hdims
        self.ydim=ydim

        self.layers=[]
        prev_hdim=self.xdim
        for hdim in self.hdims:
            self.layers.append(nn.Linear(prev_hdim,hdim))
            self.layers.append(nn.Tanh())# activation 함수 즉 활성화 함수지 
            prev_hdim=hdim
        #final Layer(without activation)
        self.layers.append(nn.Linear(prev_hdim,self.ydim,bias=True))
        #Concatenate all layers
        self.net =nn.Sequential()
        for idx,layer in enumerate(self.layers):
            layer_name='%s_%02d'%(type(layer).__name__.lower(),idx)
            self.net.add_module(layer_name,layer)
        
        self.init_param()# initialize parameters
    
    def init_param(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m,nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
    
    def forward(self,x):
        return self.net(x)
    

    print('done')


# %%
get_ipython().run_cell_magic('time', '', "lrate=1e-2\n#instantiate models\nmodel_sgd=Model(name='mlp_sgd',xdim=1,hdims=[64,64],ydim=1).to(device)\nmodel_momentum=Model(name='mlp_momentum',xdim=1,hdims=[64,64],ydim=1).to(device)\nmodel_adam=Model(name='mlp-adam',xdim=1,hdims=[64,64],ydim=1).to(device)\n#Optimizers\nloss=nn.MSELoss()\noptm_sgd=optim.SGD(\n    #fill in here\n    model_sgd.parameters(),lr=lrate\n\n)\noptm_momentum= optim.SGD(\n    #fill in here\n    model_momentum.parameters() ,lr=lrate,momentum=0.99\n\n)\noptm_adam=optim.Adam(\n    #fill in here\n    model_adam.parameters(),lr=lrate\n)\n\nprint('done')")

# %% [markdown]
# ## check Parameters
# 

# %%
np.set_printoptions(precision=3)
n_param=0
for idx,(param_name,param) in enumerate(model_sgd.named_parameters()):
    if param.requires_grad:
        param_numpy=param.detach().cpu().numpy()# to numpy array
        n_param+=len(param_numpy.reshape(-1))
        print(f'{idx} name:{param_name} shape:{param_numpy.shape}')
        print(f'val:{param_numpy.reshape(-1)[:5]}')
print(f'Total number of parameters:{format(n_param,",d")}')


# %%



# %%
MAX_ITER,BATCH_SIZE,PLOT_EVERY = 1e4,64,500

model_sgd.init_param()
model_momentum.init_param()
model_adam.init_param()

model_sgd.train()
model_momentum.train()
model_adam.train()

for it in range(int(MAX_ITER)):
    r_idx = np.random.permutation(n_data)[:BATCH_SIZE]
    batch_x,batch_y = x_torch[r_idx],y_torch[r_idx]
    
    # Update with Adam
    y_pred_adam = model_adam.forward(batch_x)
    loss_adam = loss(y_pred_adam,batch_y)
    optm_adam.zero_grad()
    loss_adam.backward()
    optm_adam.step()

    # Update with Momentum
    y_pred_momentum = model_momentum.forward(batch_x)
    loss_momentum = loss(y_pred_momentum,batch_y)
    optm_momentum.zero_grad()
    loss_momentum.backward()
    optm_momentum.step()

    # Update with SGD
    y_pred_sgd = model_sgd.forward(batch_x)
    loss_sgd = loss(y_pred_sgd,batch_y)
    optm_sgd.zero_grad()
    loss_sgd.backward()
    optm_sgd.step()
    

    # Plot
    if ((it%PLOT_EVERY)==0) or (it==0) or (it==(MAX_ITER-1)):
        with torch.no_grad():
            y_sgd_numpy = model_sgd.forward(x_torch).cpu().detach().numpy()
            y_momentum_numpy = model_momentum.forward(x_torch).cpu().detach().numpy()
            y_adam_numpy = model_adam.forward(x_torch).cpu().detach().numpy()
            
            plt.figure(figsize=(8,4))
            plt.plot(x_numpy,y_numpy,'r.',ms=4,label='GT')
            plt.plot(x_numpy,y_sgd_numpy,'g.',ms=2,label='SGD')
            plt.plot(x_numpy,y_momentum_numpy,'b.',ms=2,label='Momentum')
            plt.plot(x_numpy,y_adam_numpy,'k.',ms=2,label='ADAM')
            plt.title("[%d/%d]"%(it,MAX_ITER),fontsize=15)
            plt.legend(loc='upper right',fontsize=15)
            plt.show()

print ("Done.")


# %%
help(plt.legend)


